In [ ]:
%load_ext autoreload
%autoreload 2

# MLflow Customizable Pipeline UX Demo

This notebook illustrates how a customizable pipeline could be used in a notebook setting. Specifically, we introduced
1. Capability to override pipeline configs, step configs, and even step implementations in a live pipeline object.
2. Capability to write pipeline configs back to YAML file as source of truth when executed outside of the notebook context.

In [ ]:
from pipelines.train import MyTrainingPipeline

training = MyTrainingPipeline(profile="local")

In [ ]:
training.clean()

In [ ]:
training.inspect()

In [ ]:
training.run("ingest")

In [ ]:
training.run("evaluate")

In [ ]:
# Creates an override for register step config.
register_config = p.get_step_config("register")
register_config["model_name"] = "taxi_fare_challenger"
# Runs the register step with overriden step config.
training.run("register", step_config=register_config)

In [ ]:
# Overrides the pipeline config.
training.pipeline_config["metrics"]["primary"] = "absolute_error"
# Save the overriden step and pipeline config to train.yaml
training.save_pipeline_config()

In [ ]:
training.inspect("train")

In [ ]:
test_data = training.get_artifact("test_data")
test_data.describe()

In [ ]:
trained_model = training.get_artifact("model")
print(trained_model)

In [ ]:
# Override a step implementation in a cell.
from mlflow.pipelines.steps import IngestStep


class MyNewIngestStep(IngestStep):
    def load_file_as_dataframe(file_path: str, file_format: str):
        pass


training.steps["ingest"] = MyNewIngestStep
training.run()

In [ ]:
# Create a second pipeline for scoring.
from pipelines.scoring import MyScoringPipeline

scoring = MyScoringPipeline(profile="local")
scoring.run()